# 노인 교통 사고 발생 지역 map

In [21]:
import requests, json
# 우리가 카카오 맵으로 주소를 넘겨주면 카카오 API를 이용해서 위도 경도를 얻어옴.
def get_location(address):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
                    # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
                    # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
        headers = {"Authorization": "KakaoAK aa058a07922433baf688d53c0c06e5ea"}
        api_json = json.loads(str(requests.get(url,headers=headers).text))
        address = api_json['documents'][0]['address']
        rcd =str(address['y']) + ','+ str(address['x'])
        address_name = address['address_name']
        return rcd
    except:
        return 'r,c' # 잘못된 주소를 넣었다면 위도 경도에 r, c 값이 들어가서 쓰레기 값이 들어감

In [22]:
import pandas as pd
# 노인교통사고데이터와 법규위반 합해져있는 파일: law 을 불러오는 과정
silver_trans_csv = pd.read_csv('C:\\playdata\\23_02_01_python_mini_project\\law.csv')
silver_trans_csv

,지역구,지점명,도로명,중앙선침범,신호위반,안전거리미확보,불법유턴,과속,안전운전불이행,교차로위반,보행자보호위반,차로위반,직진우회전방해,기타,합계
0,강남구,수서역사거리,서울특별시 강남구 수서동 724 로즈데일오피스텔,0.0,2.0,5.0,0.0,0.0,6.0,1.0,0.0,1.0,0.0,1.0,16.0
1,강남구,양재전화국입구,서울특별시 강남구 도곡1동 207,0.0,4.0,2.0,0.0,0.0,11.0,1.0,0.0,0.0,0.0,0.0,18.0
2,강남구,신논현역 사거리,서울 강남구 강남대로 470,0.0,3.0,2.0,0.0,0.0,5.0,2.0,0.0,1.0,0.0,1.0,14.0
3,강남구,을지병원 앞 사거리,서울 강남구 논현동 279,0.0,3.0,2.0,0.0,0.0,7.0,4.0,0.0,1.0,0.0,0.0,17.0
4,강남구,대치우성아파트 사거리,서울 강남구 영동대로 230,0.0,7.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,중랑구,동부시장 사거리(북동),서울 중랑구 면목동 1509-2,0.0,3.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,8.0
749,중랑구,신내지하차도(하이마트상봉점),서울 중랑구 망우로 399,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,5.0
750,중랑구,면목두산5단지 앞 사거리(502동),서울 중랑구 면목동 1509-2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0
751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# 위도 경도 를 얻기 위해 law파일 데이터에서 도로명만 뽑아옴.
silver_trans_address = silver_trans_csv['도로명']
silver_trans_address

0      서울특별시 강남구 수서동 724 로즈데일오피스텔
1              서울특별시 강남구 도곡1동 207
2                 서울 강남구 강남대로 470
3                  서울 강남구 논현동 279
4                 서울 강남구 영동대로 230
                  ...            
748             서울 중랑구 면목동 1509-2
749                서울 중랑구 망우로 399
750             서울 중랑구 면목동 1509-2
751                           NaN
752                           NaN
Name: 도로명, Length: 753, dtype: object

In [24]:
# get_location 함수로 위도 경도 반환
silver_trans_latitude = []
silver_trans_longitude = []
for i in silver_trans_address:
    lat, lng = get_location(i).split(',')
    silver_trans_latitude.append(lat)
    silver_trans_longitude.append(lng)

In [25]:
# 교통 사고 발생 지역 위도 경도 데이터 프레임으로 변환: trans_address_df 
trans_address_df = pd.DataFrame({
                           '위도':silver_trans_latitude,
                           '경도':silver_trans_longitude})
exc = trans_address_df[trans_address_df["위도"].str.contains("r")].index # 아까 쓰레기값이 들어갔다면 제거하는 함수
# 교통사고 발생지역 데이터프레임에서 예외 처리된 값 제거
trans_exc = trans_address_df.drop(exc)
trans_exc.to_csv("trans_exc.csv") # 이 파일은 잘못 입력된 위도 경도 값을 제외한 정상적인 파일임.

# 교통사고 발생지역 데이터프레임 인덱스 재정의
trans_exc = trans_exc.reset_index(drop = True)

# folium으로 교통사고발생지역 지도에 마커표시 

In [27]:
import folium
map_osm = folium.Map(location=[xx[13],yy[13]],zoom_start=14)
for i in range(len(xx)):
    folium.Marker([xx[i],yy[i]], icon=folium.Icon(color='red', icon='info-sign')).add_to(map_osm)
# 지도에서 범위 지정
folium.Circle(location=[xx[13],yy[13]],popup='서울', radius=30, color="#3186CC", fill_color="#3186CC").add_to(map_osm)
#map저장
map_osm.save('Seoul_oldman_accident.html')
map_osm

# marker cluster를 사용하여 노인 교통 사고 지도에 마커 찍기

In [7]:
from ipyleaflet import Map, Marker, MarkerCluster
marker = []
m = Map(center=(37.5, 127), zoom=5)
for i in range(len(silver_trans_latitude)):
    #데이터에서 뽑아온 위도 경도를 넣고 찍어준다. 
    marker.append(Marker(location = (silver_trans_latitude[i], silver_trans_longitude[i])))

marker_cluster = MarkerCluster(markers = marker)
m.add_layer(marker_cluster)
m

Map(center=[37.5, 127], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [9]:
xx = []
yy = []
for i in range(len(trans_exc['위도'])):
    if trans_exc['위도'][i] == 0.0 or trans_exc['경도'][i] == 0.0:
        pass
    else:
        xx.append(trans_exc['위도'][i])
        yy.append(trans_exc['경도'][i])

# 노인보호구역 위경도 추출

In [13]:
import pandas as pd
silver_csv = pd.read_csv('C:\\playdata\\23_02_01_python_mini_project\\silver_zone.csv') # silver zone에 대한 주소 파일 읽어옴
silver_csv.tail()

,연번,자치구,시설명,위치,지정일
166,167.0,강동구,강동노인종합복지관,동남로71길 32-5,2018.0
167,168.0,강동구,쉼터경로당,올림픽로89길 27-23,2019.0
168,169.0,강동구,구립강암경로당,상암로15길 38-15,2020.0
169,170.0,강동구,일자산자연공원,둔촌동 산 125,2021.0
170,NaN,NaN,NaN,NaN,NaN


In [14]:
silver_address= silver_csv['위치'] # 위에서 읽어온 값 중 주소 값만 가져옴
silver_address.head()

0    율곡로19길 17-8
1      통일로14길 30
2       삼일대로 467
3       삼일대로 428
4            인의동
Name: 위치, dtype: object

In [15]:
latitude = []
longitude = []
for i in silver_address: # 위에서 얻어온 주소 값을 위도 경도로 변환
    lat, lng = get_location(i).split(',')
    latitude.append(lat)
    longitude.append(lng)

In [28]:
#위도 경도 등을 포함한 원하는 형태의 Data Frme생성
silver_address_df = pd.DataFrame({"시설명": silver_csv["시설명"],
                           '주소':silver_csv['위치'],
                           '위도':latitude,
                           '경도':longitude})


# 노인보호구역 데이터 중 쓰레기값이 들어간 경우 제외
exc = silver_address_df[silver_address_df['위도'].str.contains('r')].index
silver_address_exc = silver_address_df.drop(exc)
silver_address_exc.to_csv('trans_exc.csv')
#노인보호구역 데이터프레임 인덱스 재정의
silver_address_exc = silver_address_exc.reset_index(drop = True)
silver_address_exc

,시설명,주소,위도,경도
0,종로노인종합복지관,율곡로19길 17-8,37.5766679096376,127.006020606379
1,종로노인종합복지관 무악센터,통일로14길 30,37.575816548027,126.958022080228
2,서울노인복지센터,삼일대로 467,37.5759657913407,126.985853298403
3,락희거리,삼일대로 428,37.5727635233966,126.987902476529
4,종묘광장공원 동순라길,인의동,37.5723311920992,126.997668102599
...,...,...,...,...
165,성암장수마을 요양원,구천면로68길 46,37.5486544703182,127.142465612824
166,강동노인종합복지관,동남로71길 32-5,37.5522111843463,127.15462055421
167,쉼터경로당,올림픽로89길 27-23,37.5467987956776,127.124086561623
168,구립강암경로당,상암로15길 38-15,37.5504579238426,127.132065946177


In [29]:
x = []
y = []
for i in range(len(silver_address_exc['위도'])): 
    x.append(silver_address_exc['위도'][i])
    y.append(silver_address_exc['경도'][i])
print('x갯수: ',len(x))
print('y갯수: ',len(y))

x갯수:  170
y갯수:  170


# 노인 보호구역 

In [30]:
import folium
map_osm = folium.Map(location=[x[13],y[13]],zoom_start=14)
for i in range(len(x)):
    folium.Circle([x[i],y[i]], popup='<p style="width:120px;">%s</p>'%silver_address_exc.시설명[i], icon=folium.Icon(color='blue', icon='info-sign'),radius=300,fill='red').add_to(map_osm)
# 지도에서 범위 지정
#folium.CircleMarker(location=[x[13],y[13]],popup='서울 노인복지관', radius=300, color="#3186CC", fill_color="#3186CC").add_to(map_osm)
#map저장
map_osm.save('silver_zone.html')
map_osm

# 노인 보호구역과 사고 지역모두를 포함하는 map 생성

In [33]:
map_osm = folium.Map(location=[x[13],y[13]],zoom_start=14)

for i in range(len(x)):
    folium.Circle([x[i],y[i]], popup='<p style="width:120px;">%s</p>'%silver_address_exc.시설명[i], icon=folium.Icon(color='blue', icon='info-sign'),radius=300,fill_color='blue').add_to(map_osm)

for i in range(len(xx)):
    folium.Circle([xx[i],yy[i]],icon=folium.Icon(color='red', icon='info-sign'),radius = 50,color="red", fill_color = 'red').add_to(map_osm)
    
map_osm.save('final.html')
map_osm